In [16]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from keras import layers
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.16.0-dev20231119


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2023-11-19 17:40:10--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 2606:4700:20::681a:221, 2606:4700:20::681a:321, 2606:4700:20::ac43:4695, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|2606:4700:20::681a:221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: 'train-data.tsv'

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.1s    

2023-11-19 17:40:10 (2.92 MB/s) - 'train-data.tsv' saved [358233/358233]

--2023-11-19 17:40:10--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 2606:4700:20::681a:221, 2606:4700:20::681a:321, 2606:4700:20::ac43:4695, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|2606:4700:20::681a:221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-

In [53]:
dataset=pd.read_csv(train_file_path,  delimiter='\t')
test=pd.read_csv(test_file_path, delimiter='\t')

,ham,"ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it."
0,ham,you can never do nothing
1,ham,"now u sound like manky scouse boy steve,like! ..."
2,ham,mum say we wan to go then go... then she can s...
3,ham,never y lei... i v lazy... got wat? dat day ü ...
4,ham,in xam hall boy asked girl tell me the startin...


In [63]:
train_dataset=dataset.iloc[:,1]
train_labels=dataset.iloc[:,0]
test_dataset=test.iloc[:,1]
test_labels=test.iloc[:,0]
train_dataset.head()

0                             you can never do nothing
1    now u sound like manky scouse boy steve,like! ...
2    mum say we wan to go then go... then she can s...
3    never y lei... i v lazy... got wat? dat day ü ...
4    in xam hall boy asked girl tell me the startin...
Name: ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it., dtype: object

In [64]:
print(train_dataset.shape, train_labels.shape)
train_dataset

(4178,) (4178,)


0                                you can never do nothing
1       now u sound like manky scouse boy steve,like! ...
2       mum say we wan to go then go... then she can s...
3       never y lei... i v lazy... got wat? dat day ü ...
4       in xam hall boy asked girl tell me the startin...
                              ...                        
4173    just woke up. yeesh its late. but i didn't fal...
4174    what do u reckon as need 2 arrange transport i...
4175    free entry into our £250 weekly competition ju...
4176    -pls stop bootydelious (32/f) is inviting you ...
4177    tell my  bad character which u dnt lik in me. ...
Name: ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it., Length: 4178, dtype: object

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
X_train, X_test, y_train, y_test = train_test_split(train_dataset, train_labels, test_size=0.2, random_state=42)
vectorizer = TfidfVectorizer(max_features=4178)  # You can adjust 'max_features' based on your dataset size
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Convert labels to numerical format
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


In [66]:
X_train_tfidf


<3342x4178 sparse matrix of type '<class 'numpy.float64'>'
	with 42023 stored elements in Compressed Sparse Row format>

In [67]:
from sklearn.linear_model import LogisticRegression

model=LogisticRegression()
model.fit(X_train_tfidf,y_train_encoded)

LogisticRegression()

In [68]:
# model = keras.Sequential([
#     layers.Dense(32, activation='relu', input_shape=(X_train_tfidf.shape[1],)),
#     layers.Dense(16, activation='relu'),
#     layers.Dense(1, activation='sigmoid')
# ])
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# model.fit(X_train_tfidf, y_train_encoded, epochs=100, batch_size=32, validation_data=(X_test_tfidf, y_test_encoded), verbose=1)

In [76]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
    pred_text_encoded=vectorizer.transform([pred_text])
    p = model.predict(pred_text_encoded)
    msg=''
    if p<=0.5:
        msg='ham'
    else:
        msg='spam'
    prediction=[p,msg]
    return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[array([0]), 'ham']


In [80]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
    

    test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

    test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
    passed = True

    for msg, ans in zip(test_messages, test_answers):
        prediction = predict_message(msg)
        if prediction[1] != ans:
            passed = False

    if passed:
        print("You passed the challenge. Great job!")
    else:
        print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
